In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
from datetime import *
from sklearn import linear_model
from sklearn.preprocessing import *
from sklearn import ensemble
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import xgboost as xgb
from sklearn import metrics
from xgboost.sklearn import XGBClassifier  
from sklearn.model_selection import GridSearchCV,cross_val_score  
import matplotlib.pylab as plt  

In [2]:
#首先导入训练集数据进行线下数据集划分
train_df = pd.read_csv('chusai_train_df.csv')

regex_string = ('item_price_level_.*|item_pv_level_.*' + 
               '|shop_review_num_level_.*|shop_star_level_.*' + 
               '|context_page_id_.*' + 
               '|shop_review_positive_rate|shop_score_service|shop_score_delivery|shop_score_description')
fea = ['hour', 'user_gender_id', 'user_occupation_id', 'user_star_level', 'user_age_level', 'item_sales_level', 
       'item_brand_id', 'item_city_id', 'item_collected_level','item_price_level', 'item_pv_level', 
       'context_page_id', 'shop_review_num_level', 'shop_star_level',
      'match_category_proportion', 'match_property_proportion', 
       'predict_category_number', 'predict_property_number', 
      'isFirstCategoryIn', 'isLastCategoryIn',
       'day_hour', 'weekday',
       'lastOneHour_sameItem_count', 'lastOneHour_sameFirstCategory_count', 'lastOneHour_sameLastCategory_count',
#        'history_sameItem_count', 'history_sameFirstCategory_count', 'history_sameLastCategory_count',
       'is_history_sameItem', 'is_history_sameFirstCategory', 'is_history_sameLastCategory',
       'is_highProportion_brand', 'is_highSale_brand', 'category_number', 'property_number',
       'lastOneHour_sameBrand_count', 'lastOneHour_sameShop_count',
#        'history_sameBrand_count', 'history_sameShop_count',
       'is_history_sameBrand', 'is_history_sameShop',
#        'is_history_firstClickItem', 
       'isLastOneHour_firstClickItem',
#        'item_sales_price', 'item_sales_collected', 'item_sales_pv',
#        'item_price_collected', 'item_price_pv', 'item_collected_pv',
#        'user_gender_age', 'user_gender_occupation', 'user_gender_star', 'user_age_occupation', 'user_age_star', 'user_occupation_star',
#        'shop_review_star'
       'userItem_lastClickDeltaTime', 'userBrand_lastClickDeltaTime', 'userShop_lastClickDeltaTime', 'userFirstCategory_lastClickDeltaTime', 'userLastCategory_lastClickDeltaTime',
       'shop_item_classNumber', 'brand_item_classNumber', 'city_item_classNumber', 
       'shop_user_classNumber', 'brand_user_classNumber', 'city_user_classNumber'
      ]

drop_fea = ['item_price_level_0', 'item_price_level_1', 'shop_star_level_lowHigh', #'is_history_sameItem', 
            'shop_star_level_middle', 'shop_review_num_level_lowHigh']
def dropFeture(drop_fea, df):
    for fea in drop_fea:
        df = df.drop(fea, 1)
    return df
train_df = dropFeture(drop_fea, train_df)

train_dataset = train_df.filter(regex = regex_string)
train_dataset = pd.concat([train_dataset, train_df[fea]], axis = 1)
train_dataset_label = train_df['is_trade']
print(len(train_dataset.columns))
print(train_dataset.columns)

test_df = pd.read_csv('chusai_test_df.csv')
test_dataset = test_df.filter(regex = regex_string)
test_dataset = pd.concat([test_dataset, test_df[fea]], axis = 1)
test_dataset = dropFeture(drop_fea, test_dataset)
print(len(test_dataset.columns))
print(test_dataset.columns)

#初步先用前6天数据作为线下训练集，用第七天数据作为线下测试集
train_dataset1 = train_df[train_df.day <= 22]
train_dataset1_x = train_dataset1.filter(regex = regex_string)
train_dataset1_x = pd.concat([train_dataset1_x, train_dataset1[fea]], axis = 1)
train_dataset1_y = train_dataset1['is_trade']
test_dataset1 = train_df[train_df.day == 23]
test_dataset1_x = test_dataset1.filter(regex = regex_string)
test_dataset1_x = pd.concat([test_dataset1_x, test_dataset1[fea]], axis = 1)
test_dataset1_y = test_dataset1['is_trade']

train_df['context_timestamp'] = train_df.context_timestamp.map(lambda x: datetime.fromtimestamp(x))
test_df['context_timestamp'] = test_df.context_timestamp.map(lambda x: datetime.fromtimestamp(x))



66
Index(['shop_review_positive_rate', 'shop_score_service',
       'shop_score_delivery', 'shop_score_description', 'context_page_id_1',
       'context_page_id_2', 'context_page_id_3', 'context_page_id_4',
       'item_pv_level_low', 'shop_star_level_loss',
       'isHigh_shop_review_positive_rate', 'shop_review_positive_rate_diff',
       'isHigh_shop_score_service', 'shop_score_service_diff',
       'isHigh_shop_score_delivery', 'shop_score_delivery_diff',
       'isHigh_shop_score_description', 'shop_score_description_diff', 'hour',
       'user_gender_id', 'user_occupation_id', 'user_star_level',
       'user_age_level', 'item_sales_level', 'item_brand_id', 'item_city_id',
       'item_collected_level', 'item_price_level', 'item_pv_level',
       'context_page_id', 'shop_review_num_level', 'shop_star_level',
       'match_category_proportion', 'match_property_proportion',
       'predict_category_number', 'predict_property_number',
       'isFirstCategoryIn', 'isLastCategoryIn', 

In [3]:
class XgbModel:
    def __init__(self, feaNames=None, params={}):
        self.feaNames = feaNames
        self.params = {
            'booster':'gbtree',
            'objective': 'binary:logistic',
            'eval_metric':'logloss',
            'silent': True,
            'lambda':11,
            'eta': 0.1,
            'max_depth': 3,
            'gamma': 0.3,
            'subsample':0.9,
            'colsample_bytree': 0.65,
            'min_child_weight': 9,
            'max_delta_step': 2,
            # 'alpha':1600
        }
        for k,v in params.items():
            self.params[k] = v
        self.clf = None

    def train(self, X, y, train_size=1, test_size=0.1, verbose=True, num_boost_round=1000, early_stopping_rounds=3):
        if train_size==1:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
            X_train, y_train = X, y
        else:
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size)
        dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=self.feaNames)
        dval = xgb.DMatrix(X_test, label=y_test, feature_names=self.feaNames)
        watchlist = [(dtrain,'train'),(dval,'val')]
        clf = xgb.train(
            self.params, dtrain, 
            num_boost_round = num_boost_round, 
            evals = watchlist, 
            early_stopping_rounds = early_stopping_rounds,
            verbose_eval=verbose
        )
        self.clf = clf

    def trainCV(self, X, y, nFold=3, verbose=True, num_boost_round=1500, early_stopping_rounds=10):
        dtrain = xgb.DMatrix(X, label=y, feature_names=self.feaNames)
        cvResult = xgb.cv(
            self.params, dtrain, 
            num_boost_round = num_boost_round, 
            nfold = nFold,
            early_stopping_rounds = early_stopping_rounds,
            verbose_eval=verbose
        )
        clf = xgb.train(
            self.params, dtrain, 
            num_boost_round = cvResult.shape[0], 
        )
        self.clf = clf

    def gridSearch(self, X, y, nFold=3, verbose=1, num_boost_round=330):
        paramsGrids = {
            # 'n_estimators': [50+5*i for i in range(0,30)],
            'gamma': [i/10 for i in range(0,10)],
#             'max_depth': list(range(3,10))
#             'min_child_weight': list(range(1,10))
#              'subsample': [1-0.05*i for i in range(0,10)]
#             'colsample_bytree': [1-0.05*i for i in range(0,10)]
#             'lambda': list(range(5,15))
            # 'reg_alpha':[1000+100*i for i in range(0,20)]
            # 'max_delta_step': [0+1*i for i in range(0,8)]
        }
        gsearch = GridSearchCV(
            estimator = xgb.XGBClassifier(
                max_depth = self.params['max_depth'], 
                # gamma = self.params['gamma'],
                learning_rate = self.params['eta'],
                max_delta_step = self.params['max_delta_step'],
                min_child_weight = self.params['min_child_weight'],
                # subsample = self.params['subsample'],
                colsample_bytree = self.params['colsample_bytree'],
                silent = self.params['silent'],
                # reg_alpha = self.params['alpha'],
                n_estimators = num_boost_round
            ),
            param_grid = paramsGrids,
            scoring = 'neg_log_loss',
            cv = nFold,
            verbose = verbose,
            n_jobs = 4
        )
        gsearch.fit(X, y)
        print(pd.DataFrame(gsearch.cv_results_))
        print(gsearch.best_params_)
        exit()

    def predict(self, X):
        return self.clf.predict(xgb.DMatrix(X, feature_names=self.feaNames))

    def getFeaScore(self, show=False):
        fscore = self.clf.get_score()
        feaNames = fscore.keys()
        scoreDf = pd.DataFrame(index=feaNames, columns=['importance'])
        for k,v in fscore.items():
            scoreDf.loc[k, 'importance'] = v
        if show:
            print(scoreDf.sort_index(by=['importance'], ascending=False))
        return scoreDf

# 划分训练集和测试集
def trainTestSplit(df, splitDate=pd.to_datetime('2018-09-23'), trainPeriod=3, testPeriod=1):
    trainDf = df[(df.context_timestamp<splitDate)&(df.context_timestamp>=splitDate-timedelta(days=trainPeriod))]
    testDf = df[(df.context_timestamp>=splitDate)&(df.context_timestamp<splitDate+timedelta(days=testPeriod))]
    return (trainDf, testDf)


    

In [4]:
cv_fea = train_df.columns

# 测试模型效果
costDf = pd.DataFrame(index=fea+['cost'])
xgbModel = XgbModel(feaNames=fea)
trainDf, testDf = trainTestSplit(train_df, pd.to_datetime('2018-09-23'), trainPeriod=7)
# xgbModel.trainCV(trainDf[fea].values, trainDf['is_trade'].values)




In [5]:
xgbModel.gridSearch(trainDf[fea].values, trainDf['is_trade'].values)
# for dt in pd.date_range(start='2018-09-21', end='2018-09-24', freq='D'):
#     trainDf, testDf = trainTestSplit(train_df, dt, trainPeriod=7)
#     xgbModel.gridSearch(trainDf[fea].values, trainDf['is_trade'].values)
#     xgbModel.trainCV(trainDf[fea].values, trainDf['is_trade'].values)
#     testDf.loc[:,'predict'] = xgbModel.predict(testDf[fea].values)
#     scoreDf = xgbModel.getFeaScore()
#     scoreDf.columns = [dt.strftime('%Y-%m-%d')]
#     costDf = costDf.merge(scoreDf, how='left', left_index=True, right_index=True)
#     cost = metrics.log_loss(testDf['is_trade'].values, testDf['predict'].values)
#     costDf.loc['cost',dt.strftime('%Y-%m-%d')] = cost

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:  8.9min finished


   mean_fit_time  mean_score_time  mean_test_score  mean_train_score  \
0      67.372550         1.147380        -0.090962         -0.087461   
1      66.497854         1.222886        -0.090968         -0.087467   
2      67.568267         1.111672        -0.090987         -0.087504   
3      68.353757         1.132333        -0.090953         -0.087514   
4      67.603970         1.159069        -0.090946         -0.087503   
5      67.386241         1.132262        -0.090964         -0.087475   
6      66.471396         1.182733        -0.091022         -0.087499   
7      64.647568         1.133347        -0.091038         -0.087470   
8      64.011982         1.125286        -0.091037         -0.087420   
9      49.575414         1.054277        -0.091030         -0.087444   

  param_gamma          params  rank_test_score  split0_test_score  \
0           0  {'gamma': 0.0}                3          -0.090853   
1         0.1  {'gamma': 0.1}                5          -0.090870   
